# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
w = data[data.race=='w'].call
b = data[data.race=='b'].call

In [4]:
# number of callbacks for black-sounding names
b_calls = sum(b)
print('# of callbacks for black-sounding names: ', b_calls)

#number of callbacks for white-sounding names
w_calls = sum(w)
print('# of callbacks for white-sounding names: ', w_calls)

#percent of callbacks for all applicants
call_total = sum(data.call)
print('# of callbacks for all applicants: ', call_total)

# of callbacks for black-sounding names:  157.0
# of callbacks for white-sounding names:  235.0
# of callbacks for all applicants:  392.0


In [5]:
print(data.shape)
print(data.dtypes)
print(data.head())
print(data.tail())

(4870, 65)
id                     object
ad                     object
education                int8
ofjobs                   int8
yearsexp                 int8
honors                   int8
volunteer                int8
military                 int8
empholes                 int8
occupspecific           int16
occupbroad               int8
workinschool             int8
email                    int8
computerskills           int8
specialskills            int8
firstname              object
sex                    object
race                   object
h                     float32
l                     float32
call                  float32
city                   object
kind                   object
adid                  float32
fracblack             float32
fracwhite             float32
lmedhhinc             float32
fracdropout           float32
fraccolp              float32
linc                  float32
                       ...   
parent_emp            float32
branch_sales          float32

<div class="span5 alert alert-success">
<p>Your answers to Q1 here</p>
<p>1. What test is appropriate for this problem? Does CLT apply?</p>
</div>

The appropriate test for this problem would be a bootstrap permutation test using difference of means,
and CLT does apply because the sample size is large and the observations are independent
("Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer")

<div class="span5 alert alert-success">
<p>Your answers to Q2 here</p>
<p>2. What are the null and alternate hypotheses?</p>
</div>

Ho : There is no difference between the rate of callbacks for both black and white sounding names

Ha : There is a difference between the rate of callbacks for black-sounding and white-sounding names

<div class="span5 alert alert-success">
<p> Your answers to Q3 </p>
<p> 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches. </p>
</div>

##### Frequentist Approach

In [6]:
#calculate the standard error of mean of two samples
sem = np.sqrt(((b.std() ** 2) / len(b)) + ((w.std() ** 2) / len(w)))
dom = np.abs(np.mean(b) - np.mean(w))

#show margin of error 
print('Margin of Error: ', sem)

#create formula to calculate confidence interval
def conf_int(data_1, data_2, error, per):
    c_int = [dom - (1 + per)*sem, dom + (1 + per) * sem]
    return c_int

#show confidence interval
print('95% Confidence Interval: ', conf_int(b, w, sem, 0.95))

#perform independent samples t-test for p-value
print('T-Test: tstat = %6.3f, P-value = %6.8f' % stats.ttest_ind(w, b))

Margin of Error:  0.007784906919813793
95% Confidence Interval:  [0.016852286361423684, 0.04721342334869747]
T-Test: tstat =  4.115, P-value = 0.00003941


##### Bootstrap Approach

In [7]:
#create function for a bootstrap permutation test between the two samples
def dom_perm_test(data_1, data_2, dom, size):
    perm_replicates = np.empty(size)
    for i in range(size):
        combined = np.random.permutation(np.concatenate((data_1, data_2)))
        perm_sample_1 = combined[:len(data_1)]
        perm_sample_2 = combined[len(data_2):]
        perm_replicates[i] = np.abs(np.mean(perm_sample_1) - np.mean(perm_sample_2))
    p = np.sum(perm_replicates >= dom) / len(perm_replicates)
    return p

#show p-value
print('P-value: ',dom_perm_test(b, w, dom, 10000))

P-value:  0.0001


<div class="span5 alert alert-success">
<p> Your answers to Q4 here </p>
<p>4. Write a story describing the statistical significance in the context or the original problem.</p>
</div>

Statistical analysis through both a bootstrap approach and frequentist approach resulted in P-values well below the threshold of 0.05. Leading us to reject the null hypothesis that there is no difference between the rate of callbacks for both black and white sounding names, and accept the alternative hypothesis that there is a difference between the rate at which white sounding names and black sounding names receive callbacks on job applications. 

<div class="span5 alert alert-success">
<p> Your answers to Q5 here </p>
<p>5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</p>
</div>

Although the analysis showed race had a significant impact on the rate of callbacks for applicants, it cannot be determined at this time that it was the most imporant factor in callback success, as none of the other attributes that candidates displayed were subjected to similar procedures. I would ammend the analysis to test the significance of other features within the dataset and their relationship to callback success.